Copyright 2024 Dema-Pro Group. All rights reserved.

Use of this source code is governed by a BSD-style license that can be found in the LICENSE file.

## Imports

In [1]:
from tdw.controller import Controller
from tdw.add_ons.oculus_touch import OculusTouch
from tdw.output_data import OutputData
from tdw.add_ons.third_person_camera import ThirdPersonCamera
from tdw.tdw_utils import TDWUtils
from tdw.librarian import ModelLibrarian
from tdw.output_data import OutputData, Bounds
from scipy.spatial.transform import Rotation as R
from tdw.add_ons.image_capture import ImageCapture
from tdw.librarian import ModelLibrarian
from tdw.add_ons.keyboard import Keyboard
from tdw.add_ons.embodied_avatar import EmbodiedAvatar
from tdw.add_ons.avatar_body import AvatarBody
from tdw.add_ons.first_person_avatar import FirstPersonAvatar

import numpy as np
import os, subprocess, time

from pathlib import Path

## Helper functions

In [2]:
def get_bounds(c, obj_id, rotation):
    resp = c.communicate({"$type": "send_bounds", "frequency": "once"})
    for i in range(len(resp) - 1):
        r_id = OutputData.get_data_type_id(resp[i])
        if r_id == "boun":
            bounds = Bounds(resp[i])
            for j in range(bounds.get_num()):
                if bounds.get_id(j) == obj_id:
                    top = bounds.get_top(j)[1]
                    bottom = bounds.get_bottom(j)[1]
                    back = bounds.get_back(j)[2]
                    front = bounds.get_front(j)[2]
                    left = bounds.get_left(j)[0]
                    right = bounds.get_right(j)[0]
    init_corners = np.array(
        [
            [left, top, back],
            [left, top, front],
            [left, bottom, back],
            [left, bottom, front],
            [right, top, back],
            [right, top, front],
            [right, bottom, back],
            [right, bottom, front],
        ]
    )

    r = (
        R.from_euler(
            "xyz", [[rotation["x"], rotation["y"], rotation["z"]]], degrees=True
        )
        .as_matrix()
        .squeeze()
    )

    transformed_corners = np.matmul(r, init_corners.T).T

    return {
        "left": float(left),
        "right": float(right),
        "top": float(top),
        "bottom": float(bottom),
        "front": float(front),
        "back": float(back),
        "width": abs(float(left - right)),
        "height": abs(float(top - bottom)),
        "depth": abs(float(front - back)),
        "corners": transformed_corners.tolist(),
    }


def add_your_own_object(
    c,
    name,
    id,
    position,
    rotation={"x": 0, "y": 0, "z": 0},
    mass=5,
    color=None,
    scale={"x": 1, "y": 1, "z": 1},
    material=None,
    dynamic_friction=0.3,
    static_friction=0.3,
    bounciness=0.7,
    set_kinematic=False,
    use_gravity=True
):
    commands = []
    commands.extend(
        [
            {
                "$type": "add_object",
                "name": name,
                "url": "file:///" + models_directory + name,
                "id": id,
                "position": position,
            },
            {
                "$type": "scale_object",
                "scale_factor": scale,
                "id": id,
            },
        ]
    )
    if material:
        model_record = local_librarian.get_record(name)
        commands.extend(
            TDWUtils.set_visual_material(
                c=c,
                substructure=model_record.substructure,
                material=material,
                object_id=id,
            )
        )
    commands.append({"$type": "set_mass", "mass": mass, "id": id})
    commands.extend(
        [
            {
                "$type": "rotate_object_by",
                "angle": rotation["x"],
                "id": id,
                "axis": "pitch",
                "use_centroid": True,
            },
            {
                "$type": "rotate_object_by",
                "angle": rotation["y"],
                "id": id,
                "axis": "yaw",
                "use_centroid": True,
            },
            {
                "$type": "rotate_object_by",
                "angle": rotation["z"],
                "id": id,
                "axis": "roll",
                "use_centroid": True,
            },
        ]
    )

    commands.extend(
        [
            {
                "$type": "set_kinematic_state",
                "id": id,
                "is_kinematic": set_kinematic,
                "use_gravity": use_gravity
            },
            {
                "$type": "set_physic_material",
                "dynamic_friction": dynamic_friction,
                "static_friction": static_friction,
                "bounciness": bounciness,
                "id": id
            }
        ]
    )

    if color:
        commands.append({"$type": "set_color", "color": color, "id": id})
    c.communicate(commands)
    bounds = get_bounds(c, id, rotation)
    c.communicate([])

    return bounds

def add_tdw_object(
    c,
    name,
    id,
    position,
    rotation={"x": 0, "y": 0, "z": 0},
    mass=5,
    color=None,
    scale={"x": 1, "y": 1, "z": 1},
    material=None,
    dynamic_friction=0.3,
    static_friction=0.3,
    bounciness=0.7,
    set_kinematic=False,
    use_gravity=True,
    library="models_core.json"
):
    commands = []
    commands.extend(
        c.get_add_physics_object(
            model_name=name,
            object_id=id,
            position=position,
            rotation=rotation,
            mass=mass,
            default_physics_values = False,
            dynamic_friction=dynamic_friction,
            static_friction=static_friction,
            bounciness=bounciness,
            gravity=use_gravity,
            kinematic=set_kinematic,
            scale_factor=scale,
            library=library
        )
    )

    if color:
        commands.append({"$type": "set_color", "color": color, "id": id})

    if material:
        librarian = ModelLibrarian(library=library)
        model_record = librarian.get_record(name)
        commands.extend(
            TDWUtils.set_visual_material(
                c=c,
                substructure=model_record.substructure,
                material=material,
                object_id=id,
            )
        )

    c.communicate(commands)
    bounds = get_bounds(c, id, rotation)

    return bounds

## Controller
A controller is the Python object that communicates with the simulation application (the build). You can code up your scene in the controller and send it to the simulator for rendering. Similarly, the simulator sends the output data (including pose of every object and agent, collisions, RGB and depth maps, etc) back to the controller and you can read them out.

In [3]:
images_dir = Path("./images")
images_dir.mkdir(exist_ok=True)

videos_dir = Path("./videos")
videos_dir.mkdir(exist_ok=True)

print(f"Images will be saved to: {images_dir}")

Images will be saved to: images


In [4]:
subprocess.Popen(["~/tdw_build/TDW/TDW.app/Contents/MacOS/TDW", "-port", str(1071), "--force_glcore42", "--flip_images"], shell=True,
             stdin=None, stdout=None, stderr=None, close_fds=True)

<Popen: returncode: None args: ['~/tdw_build/TDW/TDW.app/Contents/MacOS/TDW'...>

In [5]:
c = Controller(port=1071, check_version=False, launch_build=False)

camera = ThirdPersonCamera(
    avatar_id=c.get_unique_id(),
    position={"x": 0, "y": 1.6, "z": -1.0},
    rotation={"x": 0, "y": 0, "z": 0},
    field_of_view=180,
)
c.add_ons.append(camera)

You need to launch your own build.


In [6]:
c.communicate(c.get_add_scene(scene_name="tdw_room"))
c.communicate(
    [
        TDWUtils.create_empty_room(15, 15),
        {"$type": "set_screen_size", "width": 1000, "height": 1000},
        {"$type": "set_target_framerate", "framerate": 60},
    ]
)

[b'\x00\x00\x00\x03']

In [7]:
avatar = FirstPersonAvatar(
    avatar_id="av",
    field_of_view=10,
)

c.add_ons.append(avatar)

In [ ]:
c.communicate(
    c.get_add_object(
        model_name="rh10",
        position={"x": 2, "y": 0, "z": 2},
        object_id=c.get_unique_id(),
    )
)

done = False
while not done:
    c.communicate([])
    if avatar.mouse_is_over_object and avatar.left_button_pressed:
        print(avatar.mouse_over_object_id)
    if avatar.right_button_pressed:
        done = True

c.communicate({"$type": "terminate"})